# Data Mining - Lab 1
### Team 2 - Patricia Goresen, Jeffrey Lancon, Brychan Manry, George Sturrock
#### May 27, 2018
------

## Business Understanding

Since its inception, statistical data analysis has been an integral part of the game.  Coaches, players, and baseball fans can
recite many of these stats and often use them in coaching and team roster development decisions.  Books and movies have been based on the pursuit of utilizing baseball statistics to build the ultimate baseball team. They have even coined the term Sabermetrics: The empirical analysis of baseball, especially baseball statistics that measure in-game activity.

The book; Moneyball: The Art of Winning an Unfair Game by Michael Lewis and a film based on the book, staring Brad Pitt and Jonah Hill, are about Billy Bean, General Manager of the Oakland Athletics who focuses on player/team Sabermetrics to assemble a competitive baseball team using limited funding during the 2002 and 2003 seasons. Billy Bean was able to field a competitive team with a salary budget of less than half of their larger markets competitors, by focusing on hiring undervalued players.  Despite the salary roadblock, the Athletics made the playoffs in 2002 & 2003.

The source of baseball data for this lab is the Sean Lahman Baseball Database [
http://www.seanlahman.com/baseball-archive/statistics/].  Often cited as the most complete baseball database, the data set includes twenty-seven data tables and millions of records covering most non-proprietary baseball data pertaining to offense, fielding, pitching, payroll, player demographics, team statistics, manager data and much more.  For this lab, the focus will be on team level data.  The team level data will be explored using graphical analysis using standard python libraries.  After data exploration is complete, statistical methods can be employed to determine variable correlation and/or outcome prediction using techniques such as principle component analysis (PCA) or logistic regression.  Predictive models will be verified to assure the output is useful.  For example, a logistic regression model will have its assumptions validated, competing models will be compared using best practice measurements such as Area Under the Curve (AUC), Specificity, Sensitivity and Misclassification Scores.  The outcomes of this study could provide helpful insights for baseball fans who play fantasy sports, baseball reporters and baseball managers to assist with building a more competitive team.  

------

## Data Types and Meaning

The base team data from the Lahman baseball database contains forty-eight different attributes.  The team level data begins with the 1871 season and contains team level data thorugh the conclusion of the 2017 major league baseball season.  The grain of the team data is at the season and team level.  The attributes can be divided into three general categories:  informational, offense and pitching.  Informational attributes are descriptive elements about the team.  These include unique team identifiers, league membership, division membership, post-season success indicators and home ballpark information.  Convential team offensive statistics such as total hits, at bats and home runs are also present.  Team Pitching statitistics such as earned run average (ERA), saves and strikeouts are available for analysis as well.  

In [1]:
import pandas as pd
pd.read_csv("./docobjects/DataTypeandMeaning.csv").style.hide_index()



﻿Attribute,Category,Type,DF_Type,Description
yearID,Informational,Interval,int64,The professional baseball season.
lgID,Informational,Nominal,object,The league to which the team was a member during the specified year.
teamID,Informational,Nominal,object,Unique identifier for the professional baseball team.
franchID,Informational,Nominal,object,Unique identifier for the professional baseball franchise.
divID,Informational,Nominal,object,The division within the league to which the team belonged during the specified year.
Rank,Informational,Ordinal,int64,The team's finishing rank in their division for the specified year.
G,Informational,Interval,int64,The number of games played by the team during the specified year.
Ghome,Informational,Interval,float64,The number of home games played by the team during the specified year.
W,Informational,Interval,int64,Games won by the team during the year.
L,Informational,Interval,int64,Games lost by the team during the year.


------

## Data Quality

Overall, the team level data is a high quality data set.  That said the data will be examined for missing values, unique identifiers, consistency between team names over time and any other anomalies which may arise.  The data will be subset only those records from 1970 and forward.  Professional baseball has grown tremendously since 1871.  The merger of the American League and the National League, the introduction of divisions, teams have gone out of business, new teams have been created, wild card playoffs, and evolving strategy makes data from approximatley 150 years ago likely to be too dated to be useful.  Data quality analysis will be conducted on this subset of the team data. 

In [2]:
import numpy as np
import os
import string
from matplotlib import pyplot as plt
import seaborn as sns

#Read Teams data file
teams = pd.read_csv('./sourcedata/Teams.csv')
#Select rows where year > 1969
teams2 = teams[teams.yearID > 1969]
teams2.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
1541,1970,NL,ATL,ATL,W,5,162,81.0,76,86,...,118,0.977,Atlanta Braves,Atlanta-Fulton County Stadium,1078848.0,106,106,ATL,ATL,ATL
1542,1970,AL,BAL,BAL,E,1,162,81.0,108,54,...,148,0.981,Baltimore Orioles,Memorial Stadium,1057069.0,101,98,BAL,BAL,BAL
1543,1970,AL,BOS,BOS,E,3,162,81.0,87,75,...,131,0.974,Boston Red Sox,Fenway Park II,1595278.0,108,107,BOS,BOS,BOS
1544,1970,AL,CAL,ANA,W,3,162,81.0,86,76,...,169,0.980,California Angels,Anaheim Stadium,1077741.0,96,97,CAL,CAL,CAL
1545,1970,AL,CHA,CHW,W,6,162,84.0,56,106,...,187,0.975,Chicago White Sox,Comiskey Park,495355.0,101,102,CHW,CHA,CHA


#### Identify Uniqueness Issues within Each Column

The following script shows there are no columns with the same value is each row.  There are four columns where there are only three unique values.  This is to be expected as these are binary indicators.  Missing values in these columns will be examined later in this section.  Alternatively, there are no columns where each row contains a different value.  The attendance column contain a large number of unique values.  This is to be expected as attendance can realisticaly be as low as a five digit integer and as high as a six digit integer.  In summary, there are no issues with regards to non-uniquenes or over-uniqueness in the subset team data set.  

In [3]:
#Count Unique Values for each column
teamUnique = teams2.nunique(dropna = False)
print(teamUnique)

yearID              48
lgID                 2
teamID              36
franchID            30
divID                3
Rank                 7
G                   27
Ghome               32
W                   68
L                   70
DivWin               3
WCWin                3
LgWin                3
WSWin                3
R                  393
AB                 450
H                  406
2B                 202
3B                  58
HR                 197
BB                 334
SO                 625
SB                 194
CS                  98
HBP                 88
SF                  56
RA                 416
ER                 390
ERA                258
CG                  71
SHO                 25
SV                  56
IPouts             309
HA                 431
HRA                174
BBA                325
SOA                635
E                  123
DP                 116
FP                  24
name                36
park                88
attendance        1323
BPF        

In [4]:
#Print basic stats for attendance to address any concerns about this column being overly unique
pd.set_option('display.float_format', lambda x: '%.2f' % x)
teams2.attendance.describe()

count      1324.00
mean    2049799.53
std      792309.99
min      306763.00
25%     1439223.75
50%     2001874.50
75%     2588625.00
max     4483350.00
Name: attendance, dtype: float64

#### Check for Columns with high levels of missing data

Only four columns in the teams data set have missing values.  None of these columns have missing values for every row in the subset team data set.  The missing values in the "DivWin", "LgWin" and "WSWin" columns are due to the baseball players strike in 1994 which caused the season to end prematurely.  There were no post season games in 1994.  The 640 missing values in "WCWin" are due to the wild card playoff system being introduced in the 1995 season.  The missing values are accurate missing values due to the 1994 players strike and the introduction of the wild card post season format in 1995.  

In [5]:
#Identify any columns with no values
def highlight_high(data, ul ,color='red'):
    '''
    check if value is over a threshold
    '''
    attr = 'background-color: {}'.format(color)
    if data.str.contains("%").any():
        data = pd.to_numeric(data.str.strip("%")) / 100
    
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_high = data > ul
        return [attr if v else '' for v in is_high]
    else:  # from .apply(axis=None)
        is_high = data > ul
        return pd.DataFrame(np.where(is_high, attr, ''),index=data.index, columns=data.columns)

teamNullCols = teams2.isnull().sum()
teamNullCols = teamNullCols[teamNullCols > 0]
teamNullCols = pd.DataFrame({
    'n Missing' : teamNullCols,
    '% Missing' : ((teamNullCols / teams2.shape[0])*100).round(2)#.astype(str) + "%"
})#.style.apply(highlight_high, ul=.3, subset = ['% Missing'])

cm = sns.light_palette("red", as_cmap=True)
teamNullCols.style.background_gradient(cmap=cm)


,n Missing,% Missing
DivWin,28,2.11
WCWin,640,48.34
LgWin,28,2.11
WSWin,28,2.11


#### Additional Data Quality Checks???

Please insert any additional details.

------

## New Features

The team level data set presents opportunities to augment the data with common hitting and pitching ratios as well as merging data from additional tables.  In this section, payroll data will be summarized and merged with the Team data table.  New features will be created based on data present in the team table.  WHIP (walk and hits per innings pitched), strikeout to walk ratio, team batting average, slugging percentage, strikeout per at bat, on base percentage and OPS (on base plus slugging percentage) will all be added as new columns for the team data table.  Finally, a yes/no playoff flag will be added to show if the team made the playoffs each season.  

| ﻿Attribute | Category      | Type     | DF_Type | Description                                             |
|-----------|---------------|----------|---------|---------------------------------------------------------|
| Payroll   | Informational | Interval | float64 | Total team payroll.                                     |
| WHIP      | Pitching      | Ratio    | float64 | Walk and hits per innings pitched.                      |
| KBB       | Pitching      | Ratio    | float64 | Strikout to walk ratio.                                 |
| Bavg      | Offense       | Ratio    | float64 | Team batting average.                                   |
| Slug      | Offense       | Ratio    | float64 | Team slugging percentage.                               |
| KAB       | Offense       | Ratio    | float64 | Team strikeouts per at bat.                             |
| OBP       | Offense       | Ratio    | float64 | Team on base percentage.                                |
| OPS       | Offense       | Ratio    | float64 | Team on base + slugging percentage.                     |
| Playoff   | Informational | Nominal  | object  | Binary indicator showing is the team made the playoffs. |

#### Create Informational Columns

In [6]:
#Create Payroll Column

#Read salary data table
salaries = pd.read_csv("./sourcedata/Salaries.csv")
salaries.info()

#Summarize Salaries Table by year and team
summarisedSal = pd.DataFrame(salaries.groupby(['yearID', 'teamID']).sum().reset_index())
print(summarisedSal)

#Merge Summarized Salaries with Teams table.
teams2Plus = pd.merge(left=teams2, right=summarisedSal, how='left', on=['yearID', 'teamID'])
teams2Plus.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26428 entries, 0 to 26427
Data columns (total 5 columns):
yearID      26428 non-null int64
teamID      26428 non-null object
lgID        26428 non-null object
playerID    26428 non-null object
salary      26428 non-null int64
dtypes: int64(2), object(3)
memory usage: 1.0+ MB
     yearID teamID     salary
0      1985    ATL   14807000
1      1985    BAL   11560712
2      1985    BOS   10897560
3      1985    CAL   14427894
4      1985    CHA    9846178
5      1985    CHN   12702917
6      1985    CIN    8359917
7      1985    CLE    6551666
8      1985    DET   10348143
9      1985    HOU    9993051
10     1985    KCA    9321179
11     1985    LAN   10967917
12     1985    MIN    5764821
13     1985    ML4   11284107
14     1985    MON    9470166
15     1985    NYA   14238204
16     1985    NYN   10834762
17     1985    OAK    9058606
18     1985    PHI   10124966
19     1985    PIT    9227500
20     1985    SDN   11036583
21     1985   

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro,salary
0,1970,NL,ATL,ATL,W,5,162,81.00,76,86,...,0.98,Atlanta Braves,Atlanta-Fulton County Stadium,1078848.00,106,106,ATL,ATL,ATL,nan
1,1970,AL,BAL,BAL,E,1,162,81.00,108,54,...,0.98,Baltimore Orioles,Memorial Stadium,1057069.00,101,98,BAL,BAL,BAL,nan
2,1970,AL,BOS,BOS,E,3,162,81.00,87,75,...,0.97,Boston Red Sox,Fenway Park II,1595278.00,108,107,BOS,BOS,BOS,nan
3,1970,AL,CAL,ANA,W,3,162,81.00,86,76,...,0.98,California Angels,Anaheim Stadium,1077741.00,96,97,CAL,CAL,CAL,nan
4,1970,AL,CHA,CHW,W,6,162,84.00,56,106,...,0.97,Chicago White Sox,Comiskey Park,495355.00,101,102,CHW,CHA,CHA,nan


In [7]:
#Create Playoff Indicator
teams2Plus['Playoff'] = teams2Plus.DivWin

if (teams2Plus.WCWin.any() == 'Y'):
    teams2Plus.Playoff = 'Y'

#### Create Pitching Columns

In [8]:
#Create WHIP - (Walks + Hits) / Innings Pitched
teams2Plus['WHIP'] = (teams2Plus.BBA + teams2Plus.HA) / (teams2Plus.IPouts / 3)

#Create Strikeout to Walk Ratio (KBB)
teams2Plus['KBB'] = (teams2Plus.SOA / teams2Plus.BBA)

#### Create Offense Columns

In [9]:
#Strikeouts per At Bat (KAB)
teams2Plus['KAB'] = (teams2Plus.SO / teams2Plus.AB)

#Batting Average (Bavg)
teams2Plus['Bavg'] = (teams2Plus.H / teams2Plus.AB)

#Slugging Percentage (Slug)
##(Singles + Doubles*2 + Triples*3 + HR*4)/AB
teams2Plus['Slug'] = ((teams2Plus.H - teams2Plus['2B'] - teams2Plus['3B'] - teams2Plus.HR) + (teams2Plus['2B']*2) + (teams2Plus['3B']*3) + (teams2Plus.HR*4)) / teams2Plus.AB

#On Base Percentage (OBP)
##OBP = (Hits + Walks + Hit by Pitch) / (At Bats + Walks + Hit by Pitch + Sacrifice Flies
teams2Plus['OBP'] = (teams2Plus.H + teams2Plus.BB + teams2Plus.HBP) / (teams2Plus.AB + teams2Plus.BB + teams2Plus.HBP + teams2Plus.SF)

#On Base Percentage Plus Slugging (OPS)
teams2Plus['OPS'] = teams2Plus.Slug + teams2Plus.OBP

teams2Plus.tail()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,teamIDretro,salary,Playoff,WHIP,KBB,KAB,Bavg,Slug,OBP,OPS
1319,2017,NL,SLN,STL,C,3,162,81.00,83,79,...,SLN,nan,N,1.30,2.74,0.25,0.26,0.43,0.33,0.76
1320,2017,AL,TBA,TBD,E,3,162,81.00,80,82,...,TBA,nan,N,1.26,2.69,0.28,0.24,0.42,0.32,0.74
1321,2017,AL,TEX,TEX,W,4,162,81.00,78,84,...,TEX,nan,N,1.40,1.98,0.27,0.24,0.43,0.32,0.75
1322,2017,AL,TOR,TOR,E,4,162,81.00,76,86,...,TOR,nan,N,1.37,2.50,0.24,0.24,0.41,0.31,0.72
1323,2017,NL,WAS,WSN,E,1,162,81.00,97,65,...,WAS,nan,Y,1.24,2.94,0.24,0.27,0.45,0.33,0.78
